In [14]:
# coding:utf-8  
'''
@author: Jason.F
@data: 2019.07.15
@function: Implementing SVDBias with surprise lirbray
           Dataset: KnowledgeBase-cc 
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the KnowledgeBase dataset.
trainset = pd.read_csv("/data/fjsdata/ctKngBase/kbcc_trainset.csv", sep='|', low_memory=False)
#trainset['num']=trainset['num'].apply(lambda x: 1 if float(x)>0.0 else 0)
print ('Trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
#testset includes 100 items for every user, one item is positive and other 99 is negtive items.
testset = pd.read_csv("/data/fjsdata/ctKngBase/kbcc_testset.csv", sep='|', low_memory=False)
#testset['num']=testset['num'].apply(lambda x: 1 if float(x)>0.0 else 0)
print ('Testset shape is:%d rows and %d columns'%(testset.shape[0],testset.shape[1]))
csrNum = trainset['csr'].max()+1
keNum = trainset['ke'].max()+1
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % 
      (trainset.shape[0], csrNum, keNum, trainset.shape[0]/(csrNum*keNum)) )

#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 200))
spdata = sp.Dataset.load_from_df(trainset,reader)
trainset = spdata.build_full_trainset()
testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

Trainset shape is:2547452 rows and 3 columns
Testset shape is:1021600 rows and 3 columns
Dataset Statistics: Interaction = 2547452, User = 10216, Item = 96324, Sparsity = 0.0026
  K               HR@10             NDCG@10
  8            0.196359            0.150287
 16            0.178837            0.135577
 32            0.162686            0.126575
 64            0.152408            0.119283


In [11]:
# coding:utf-8  
'''
@author: Jason.F
@data: 2019.07.15
@function: Implementing SVDBias with surprise lirbray
           Dataset: Pinterest-20
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the dataset.
def load_dataset():
    train_data = pd.read_csv("/data/fjsdata/ctKngBase/ml/pinterest-20.train.rating", \
                             sep='\t', header=None, names=['user', 'item', 'rating'], \
                             usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2 :np.float})
    
    test_data = []
    with open("/data/fjsdata/ctKngBase/ml/pinterest-20.test.negative", 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1], 1])#one postive item
            for i in arr[1:]:
                test_data.append([u, int(i), 0]) #99 negative items
            line = fd.readline()
    return train_data, test_data

train_data,test_set = load_dataset()
#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(train_data,reader)
trainset = spdata.build_full_trainset()
#testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(test_set)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

  K               HR@10             NDCG@10
  8            0.496965            0.496965
 16            0.500299            0.500299
 32            0.495533            0.495533
 64            0.497599            0.497599


In [8]:
# coding:utf-8  
'''
@author: Jason.F
@data: 2019.07.15
@function: Implementing SVDBias with surprise lirbray
           Dataset: Movielen-1m
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the dataset.
def load_dataset():
    train_data = pd.read_csv("/data/fjsdata/ctKngBase/ml/ml-1m.train.rating", \
                             sep='\t', header=None, names=['user', 'item', 'rating'], \
                             usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2 :np.float})
    
    test_data = []
    with open("/data/fjsdata/ctKngBase/ml/ml-1m.test.negative", 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1], 1])#one postive item
            for i in arr[1:]:
                test_data.append([u, int(i), 0]) #99 negative items
            line = fd.readline()
    return train_data, test_data

train_data,test_set = load_dataset()
#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 5))
spdata = sp.Dataset.load_from_df(train_data,reader)
trainset = spdata.build_full_trainset()
#testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(test_set)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

  K               HR@10             NDCG@10
  8            0.260430            0.127373
 16            0.260430            0.127286
 32            0.260927            0.127980
 64            0.259603            0.126590


In [15]:
'''
@author: Jason.F
@data: 2019.07.13
@function: Implementing SVDBias with surprise lirbray
           Dataset: Pinterest-20
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the dataset.
def load_dataset():
    train_data = pd.read_csv("/data/fjsdata/ctKngBase/ml/pinterest-20.train.rating", \
                             sep='\t', header=None, names=['user', 'item', 'rating'], \
                             usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 1:np.float})
    #user_num = train_data['user'].max() + 1
    #item_num = train_data['item'].max() + 1 
    #train_data = train_data.values.tolist()
    
    test_data = []
    with open("/data/fjsdata/ctKngBase/ml/pinterest-20.test.negative", 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1], 1])#one postive item
            for i in arr[1:]:
                test_data.append([u, int(i), 0]) #99 negative items
            line = fd.readline()
    return train_data, test_data
train_data,test_set = load_dataset()
#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(train_data,reader)
trainset = spdata.build_full_trainset()
#testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            #if positem == -1: positem=iid #one positive item in first
            #scorelist.append([iid,est])
            if (ture_r+1)>0.0:#one positive item
                scorelist.append([iid,est])
                positem = iid 
            else:# 99 negative items
                scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

  K               HR@10             NDCG@10
  8            0.963097            0.678305
 16            0.959769            0.677092
 32            0.961042            0.677676
 64            0.958888            0.674344


In [14]:
'''
@author: Jason.F
@data: 2019.07.13
@function: Implementing SVDBias with surprise lirbray
           Dataset: Movielen-1m
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the dataset.
def load_dataset():
    train_data = pd.read_csv("/data/fjsdata/ctKngBase/ml/ml-1m.train.rating", \
                             sep='\t', header=None, names=['user', 'item', 'rating'], \
                             usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 1:np.float})
    #user_num = train_data['user'].max() + 1
    #item_num = train_data['item'].max() + 1 
    #train_data = train_data.values.tolist()
    
    test_data = []
    with open("/data/fjsdata/ctKngBase/ml/ml-1m.test.negative", 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1], 1])#one postive item
            for i in arr[1:]:
                test_data.append([u, int(i), 0]) #99 negative items
            line = fd.readline()
    return train_data, test_data
train_data,test_set = load_dataset()
#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(train_data,reader)
trainset = spdata.build_full_trainset()
#testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            #if positem == -1: positem=iid #one positive item in first
            #scorelist.append([iid,est])
            if (ture_r+1)>0.0:#one positive item
                scorelist.append([iid,est])
                positem = iid 
            else:# 99 negative items
                scorelist.append([iid,est])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

  K               HR@10             NDCG@10
  8            1.000000            1.000000
 16            1.000000            1.000000
 32            1.000000            1.000000
 64            1.000000            1.000000


In [7]:
'''
@author: Jason.F
@data: 2019.07.13
@function: Implementing SVDBias with surprise lirbray
           Dataset: KnowledgeBase-cc 
           Evaluating by hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq
import surprise as sp

#1. loading the KnowledgeBase dataset.
trainset = pd.read_csv("/data/fjsdata/ctKngBase/kbcc_trainset.csv", sep='|', low_memory=False)
trainset['num']=trainset['num'].apply(lambda x: 1 if float(x)>0.0 else 0)
print ('Trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
#testset includes 100 items for every user, one item is positive and other 99 is negtive items.
testset = pd.read_csv("/data/fjsdata/ctKngBase/kbcc_testset.csv", sep='|', low_memory=False)
testset['num']=testset['num'].apply(lambda x: 1 if float(x)>0.0 else 0)
print ('Testset shape is:%d rows and %d columns'%(testset.shape[0],testset.shape[1]))
csrNum = trainset['csr'].max()+1
keNum = trainset['ke'].max()+1
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % 
      (trainset.shape[0], csrNum, keNum, trainset.shape[0]/(csrNum*keNum)) )

#2. Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(trainset,reader)
trainset = spdata.build_full_trainset()
testset = np.array(testset).tolist()

#3.training and evaluating 
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0
print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#iterations epoches
    algo = sp.SVD(n_factors=K, n_epochs=20, lr_all=0.001, reg_all=0.01 )#NMF,SVDpp
    algo.fit(trainset)
    #print (algo.predict(str(1),str(1), r_ui=0, verbose=True)) 
    predictions = algo.test(testset)#testset include one positive and 99 negtive sample of every user.
    user_iid_true_est = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_iid_true_est[uid].append((iid, true_r, est))
    hits = []
    ndcgs = []
    for uid, iid_ratings in user_iid_true_est.items():
        # Sort user ratings by estimated value
        #iid_ratings.sort(key=lambda x: x[2], reverse=True) #sorted by est
        scorelist = []
        positem = -1
        for iid, ture_r, est in iid_ratings:
            if positem == -1: positem=iid #one positive item in first
            scorelist.append([iid,est])
            '''
            if (ture_r+1)>0.0:#one positive item
                scorelist.append([iid,est])
                positem = iid 
            else:# 99 negative items
                scorelist.append([iid,est])
            '''
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, positem)
        hits.append(hr)
        ndcg = getNDCG(ranklist, positem)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

Trainset shape is:2547452 rows and 3 columns
Testset shape is:1021600 rows and 3 columns
Dataset Statistics: Interaction = 2547452, User = 10216, Item = 96324, Sparsity = 0.0026
  K               HR@10             NDCG@10
  8            0.500098            0.500098
 16            0.490407            0.490407
 32            0.501077            0.501077
 64            0.498727            0.498727


In [ ]:
'''
@author: Jason.F
@data: 2019.07.12
@function: Implementation: SVDBias 
           Datatset: Movielen-1m 
           Evaluation: hitradio,ndcg
'''
import pandas as pd
import minpy.numpy as np
import math
from collections import defaultdict
import heapq

#1.Loading the  MovienLen dataset, ml-1m
def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            ratingList.append([user, item])
            line = f.readline()
    return ratingList
def load_negative_file_as_list(filename):
    negativeList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList
def load_rating_file_as_matrix(filename):
    #Read .rating file and Return dok matrix.
    #The first line of .rating file is: num_users\t num_items
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    #mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    mat = np.zeros((num_users+1, num_items+1))
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0.0): mat[user, item] = 1.0
            line = f.readline()    
    return mat
trainMatrix = load_rating_file_as_matrix("/data/fjsdata/ctKngBase/ml/ml-1m.train.rating")
testRatings = load_rating_file_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.test.rating")
testNegatives = load_negative_file_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.test.negative")
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
      (len(trainMatrix[np.where(trainMatrix!= 0)]),trainMatrix.shape[0],trainMatrix.shape[1],\
       len(trainMatrix[np.where(trainMatrix!= 0)])/(trainMatrix.shape[0]*trainMatrix.shape[1]) ))

#2. SVDBias class
class SVDBias():
    
    def __init__(self, R, num_ng=4):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - num_ng (int)  : number of negative items
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.num_ng = num_ng
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        '''
        #smapling the negative items
        for x in self.samples:
            u = x[0]
            for t in range(self.num_ng):
                j = np.random.randint(self.num_items)
                #while (u, j) in self.R:
                while self.R[u, j] > 0:
                    j = np.random.randint(self.num_items)
                self.samples.append([u, j, 0])
        '''

    def train(self, K, alpha=0.001, beta=0.01, epochs=20):
        '''
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        - K (int)       : number of latent dimensions
        -epochs(int)    : number of iterations
        '''
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.epochs = epochs
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
               
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.epochs):
            np.random.shuffle(self.samples)
            self.sgd()
            #if (i+1) % 10 == 0:
            #    mse = self.mse()
            #    print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return self.full_matrix()

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
    
#3. Training and Evaluating
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
mdl = SVDBias(R=trainMatrix, num_ng=4)# K is latent factors
for K in [8,16,32,64]:#latent factors
    nR = mdl.train(K=K, alpha=0.001, beta=0.01, epochs=20)
    hits = []
    ndcgs = []
    for u, i in testRatings:
        scorelist= [ [ni,nR[u,ni]] for ni in testNegatives[u]]
        scorelist.append([i,nR[u,i]])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, i)
        hits.append(hr)
        ndcg = getNDCG(ranklist, i)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

  K               HR@10             NDCG@10


In [ ]:
'''
@author: Jason.F
@data: 2019.07.12
@function: Implementation: SVDBias 
           Datatset: Pinterest-20 
           Evaluation: hitradio,ndcg
'''
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import heapq

#1.Loading the  MovienLen dataset, ml-1m
def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            ratingList.append([user, item])
            line = f.readline()
    return ratingList
def load_negative_file_as_list(filename):
    negativeList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList
def load_rating_file_as_matrix(filename):
    #Read .rating file and Return dok matrix.
    #The first line of .rating file is: num_users\t num_items
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    #mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    mat = np.zeros((num_users+1, num_items+1))
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0.0): mat[user, item] = 1.0
            line = f.readline()    
    return mat
trainMatrix = load_rating_file_as_matrix("/data/fjsdata/ctKngBase/ml/pinterest-20.train.rating")
testRatings = load_rating_file_as_list("/data/fjsdata/ctKngBase/ml/pinterest-20.test.rating")
testNegatives = load_negative_file_as_list("/data/fjsdata/ctKngBase/ml/pinterest-20.test.negative")
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
      (len(trainMatrix[np.where(trainMatrix!= 0)]),trainMatrix.shape[0],trainMatrix.shape[1],\
       len(trainMatrix[np.where(trainMatrix!= 0)])/(trainMatrix.shape[0]*trainMatrix.shape[1]) ))

#2. SVDBias class
class SVDBias():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            #if (i+1) % 10 == 0:
            #    mse = self.mse()
            #    print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return self.full_matrix()

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
    
#3. Training and Evaluating
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

print ("%3s%20s%20s" % ('K','HR@10', 'NDCG@10'))
for K in [8,16,32,64]:#latent factors
    mdl = SVDBias(R=trainMatrix, K=K, alpha=0.001, beta=0.01, iterations=20)# K is latent factors
    nR = mdl.train()
    hits = []
    ndcgs = []
    for u, i in testRatings:
        scorelist= [ [ni,nR[u,ni]] for ni in testNegatives[u]]
        scorelist.append([i,nR[u,i]])
        map_item_score = {}
        for item, rate in scorelist: #turn dict
            map_item_score[item] = rate
        ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
        hr = getHitRatio(ranklist, i)
        hits.append(hr)
        ndcg = getNDCG(ranklist, i)
        ndcgs.append(ndcg)
    hitratio,ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print ("%3d%20.6f%20.6f" % (K, hitratio, ndcg))

Dataset Statistics: Interaction = 1408394, User = 55187, Item = 9916, Sparsity = 0.0026
  K               HR@10             NDCG@10
  8            0.092232            0.041091
 16            0.094678            0.042397
 32            0.094080            0.042738


In [54]:
'''
@author: Jason.F
@data: 2019.07.11
@function: Implementation: SVDBias 
           Datatset: Movielen Dataset(ml-1m) 
           Evaluation: rmse,hitradio,ndcg
'''
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import heapq

#1.Loading the  MovienLen dataset, ml-1m
def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            #if (rating > 0): rating=1.0
            ratingList.append([user, item, rating])
            line = f.readline()
    return ratingList
def load_negative_file_as_list(filename):
    negativeList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList
def load_rating_file_as_matrix(filename):
    #Read .rating file and Return dok matrix.
    #The first line of .rating file is: num_users\t num_items
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    #mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    mat = np.zeros((num_users+1, num_items+1))
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            #if (rating > 0): mat[user, item] = 1.0
            mat[user][item] = rating
            line = f.readline()    
    return mat
trainMatrix = load_rating_file_as_matrix("/data/fjsdata/ctKngBase/ml/ml-1m.train.rating")
testRatings = load_rating_file_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.test.rating")
testNegatives = load_negative_file_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.test.negative")
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
      (len(trainMatrix[np.where(trainMatrix!= 0)]),trainMatrix.shape[0],trainMatrix.shape[1],\
       len(trainMatrix[np.where(trainMatrix!= 0)])/(trainMatrix.shape[0]*trainMatrix.shape[1]) ))

#2. SVDBias class
class SVDBias():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            #if (i+1) % 10 == 0:
            #    mse = self.mse()
            #    print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return self.full_matrix()

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
    
#3. Training and Evaluating
def getMSE(TRating,PRating):
    error = TRating-PRating
    return error*error

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

print ("%3s%20s%20s%20s%20s" % ('K','Iterations','RMSE', 'HitRatio', 'NDCG'))
for K in [8,16,32,64]:#latent factors
    for iterations in [20,50,100]:#iterations epoches
        mdl = SVDBias(R=trainMatrix, K=K, alpha=0.001, beta=0.01, iterations=iterations)# K is latent factors
        nR = mdl.train()
        mses = []
        hits = []
        ndcgs = []
        for u, i, r in testRatings:
            mses.append(getMSE(r,nR[u,i]))
            scorelist= [ [ni,nR[u,ni]] for ni in testNegatives[u]]
            scorelist.append([i,nR[u,i]])
            map_item_score = {}
            for item, rate in scorelist: #turn dict
                map_item_score[item] = rate
            ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
            hr = getHitRatio(ranklist, i)
            hits.append(hr)
            ndcg = getNDCG(ranklist, i)
            ndcgs.append(ndcg)
        rmse,hitratio,ndcg = math.sqrt(sum(mses) / len(mses)) ,np.array(hits).mean(), np.array(ndcgs).mean()
        print ("%3d%20d%20.6f%20.6f%20.6f" % (K, iterations, rmse, hitratio, ndcg))

Dataset Statistics: Interaction = 994169, User = 6040, Item = 3706, Sparsity = 0.0444
  K          Iterations                RMSE            HitRatio                NDCG
  8                  20            0.958511            0.258113            0.126652
  8                  50            0.944725            0.242715            0.120635
  8                 100            0.916240            0.258940            0.135366
 16                  20            0.957442            0.257616            0.126229
 16                  50            0.942338            0.239735            0.118111
 16                 100            0.915297            0.260265            0.137506
 32                  20            0.957337            0.260762            0.126890
 32                  50            0.944851            0.242715            0.117821
 32                 100            0.908712            0.266391            0.140854
 64                  20            0.957565            0.260927           

In [55]:
'''
@author: Jason.F
@data: 2019.07.11
@function: Implementation: SVDBias 
           Datatset: Pinterest (pinterest-20)
           Evaluation: rmse,hitradio,ndcg
'''
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import heapq

#1.Loading the  MovienLen dataset, ml-1m
def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            #if (rating > 0): rating=1.0
            ratingList.append([user, item, rating])
            line = f.readline()
    return ratingList
def load_negative_file_as_list(filename):
    negativeList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList
def load_rating_file_as_matrix(filename):
    #Read .rating file and Return dok matrix.
    #The first line of .rating file is: num_users\t num_items
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    #mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    mat = np.zeros((num_users+1, num_items+1))
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            #if (rating > 0): mat[user, item] = 1.0
            mat[user][item] = rating
            line = f.readline()    
    return mat
trainMatrix = load_rating_file_as_matrix("/data/fjsdata/ctKngBase/ml/pinterest-20.train.rating")
testRatings = load_rating_file_as_list("/data/fjsdata/ctKngBase/ml/pinterest-20.test.rating")
testNegatives = load_negative_file_as_list("/data/fjsdata/ctKngBase/ml/pinterest-20.test.negative")
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
      (len(trainMatrix[np.where(trainMatrix!= 0)]),trainMatrix.shape[0],trainMatrix.shape[1],\
       len(trainMatrix[np.where(trainMatrix!= 0)])/(trainMatrix.shape[0]*trainMatrix.shape[1]) ))

#2. SVDBias class
class SVDBias():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            #if (i+1) % 10 == 0:
            #    mse = self.mse()
            #    print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return self.full_matrix()

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
    
#3. Training and Evaluating
def getMSE(TRating,PRating):
    error = TRating-PRating
    return error*error

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

print ("%3s%20s%20s%20s%20s" % ('K','Iterations','RMSE', 'HitRatio', 'NDCG'))
for K in [8,16,32,64]:#latent factors
    for iterations in [20,50,100]:#iterations epoches
        mdl = SVDBias(R=trainMatrix, K=K, alpha=0.001, beta=0.01, iterations=iterations)# K is latent factors
        nR = mdl.train()
        mses = []
        hits = []
        ndcgs = []
        for u, i, r in testRatings:
            mses.append(getMSE(r,nR[u,i]))
            scorelist= [ [ni,nR[u,ni]] for ni in testNegatives[u]]
            scorelist.append([i,nR[u,i]])
            map_item_score = {}
            for item, rate in scorelist: #turn dict
                map_item_score[item] = rate
            ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
            hr = getHitRatio(ranklist, i)
            hits.append(hr)
            ndcg = getNDCG(ranklist, i)
            ndcgs.append(ndcg)
        rmse,hitratio,ndcg = math.sqrt(sum(mses) / len(mses)) ,np.array(hits).mean(), np.array(ndcgs).mean()
        print ("%3d%20d%20.6f%20.6f%20.6f" % (K, iterations, rmse, hitratio, ndcg))

Dataset Statistics: Interaction = 1408394, User = 55187, Item = 9916, Sparsity = 0.0026
  K          Iterations                RMSE            HitRatio                NDCG
  8                  20            0.038986            0.089623            0.040215
  8                  50            0.033846            0.075598            0.033086
  8                 100            0.027578            0.066827            0.028344
 16                  20            0.014740            0.094533            0.042753
 16                  50            0.013429            0.087140            0.038314
 16                 100            0.011655            0.076214            0.033107
 32                  20            0.005235            0.096309            0.043037
 32                  50            0.004900            0.089441            0.039722
 32                 100            0.004364            0.078406            0.034508
 64                  20            0.001858            0.096581         

In [50]:
'''
@author: Jason.F
@data: 2019.07.10
@function: Implementing SVDBias and Setting KnowledgeBase as baseline by rmse,hitradio,ndcg
'''
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import heapq

#1. loading the KnowledgeBase dataset.
trainset = pd.read_csv("/data/fjsdata/ctKngBase/trainset.csv", sep='|', low_memory=False)
print ('Trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
#testset includes 100 items for every user, one item is positive and other 99 is negtive items.
testset = pd.read_csv("/data/fjsdata/ctKngBase/testset.csv", sep='|', low_memory=False)
print ('Testset shape is:%d rows and %d columns'%(testset.shape[0],testset.shape[1]))
csrNum = trainset['csr'].max()+1
keNum = trainset['ke'].max()+1
print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % 
      (trainset.shape[0], csrNum, keNum, trainset.shape[0]/(csrNum*keNum)) )

#2. SVDBias class
class SVDBias():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            #if (i+1) % 10 == 0:
            #    mse = self.mse()
            #    print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return self.full_matrix()

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
        
#3. training and evaluating 
def getMSE(TRating,PRating):
    error = TRating-PRating
    return error*error

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

R = np.zeros((csrNum, keNum))
for index, row in trainset.iterrows(): 
    R[int(row['csr'])][int(row['ke'])] = float(row['num'])
print ("%3s%20s%20s%20s%20s" % ('K','Iterations','RMSE', 'HitRatio', 'NDCG'))
for K in [2,50,100,200]:#latent factors
    for iterations in [2,20,50,100]:#iterations epoches
        mdl = SVDBias(R=R, K=K, alpha=0.001, beta=0.01, iterations=iterations)# K is latent factors
        nR = mdl.train()
        mses = []
        hits = []
        ndcgs = []
        list_csr = list(set(np.array(testset['csr']).tolist()))
        for csr in list_csr:
            csrset = testset[testset['csr']==csr]
            scorelist = []
            positem = 0
            for u, i, r in np.array(csrset).tolist():   
                if float(r)>0.0:#one positive item
                    mses.append(getMSE(float(r),nR[int(u),int(i)]))
                    scorelist.append([int(i),nR[int(u),int(i)]])
                    positem = int(i) 
                else:# 99 negative items
                    scorelist.append([int(i),nR[int(u),int(i)]])
            map_item_score = {}
            for item, rate in scorelist: #turn dict
                map_item_score[item] = rate
            ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
            hr = getHitRatio(ranklist, positem)
            hits.append(hr)
            ndcg = getNDCG(ranklist, positem)
            ndcgs.append(ndcg)
        rmse,hitratio,ndcg = math.sqrt(sum(mses) / len(mses)) ,np.array(hits).mean(), np.array(ndcgs).mean()
        print ("%3d%20d%20.6f%20.6f%20.6f" % (K, iterations, rmse, hitratio, ndcg))

Trainset shape is:2547452 rows and 3 columns
Testset shape is:1021600 rows and 3 columns
Dataset Statistics: Interaction = 2547452, User = 10216, Item = 96324, Sparsity = 0.0026
  K          Iterations                RMSE            HitRatio                NDCG
  2                   2            0.268543            0.077721            0.034852
  2                  20            0.156521            0.073218            0.038080
  2                  50            0.122179            0.101899            0.058513
  2                 100            0.098171            0.126175            0.075000
 50                   2            0.055590            0.309417            0.250908
 50                  20            0.057026            0.385180            0.296333
 50                  50            0.055933            0.431774            0.316457
 50                 100            0.054602            0.462706            0.326995
100                   2            0.055675            0.338391   

In [3]:
#SVDBias Implementation
#https://github.com/albertauyeung/matrix-factorization-in-python
import numpy as np
class IRTMF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty entries in a matrix.     
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

    
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])
# Perform training and obtain the user and item matrices 
mf = IRTMF(R, K=2, alpha=0.1, beta=0.01, iterations=20)
training_process = mf.train()
print(mf.P)
print(mf.Q)
print(mf.full_matrix())

Iteration: 10 ; error = 0.4114
Iteration: 20 ; error = 0.0391
[[-1.19898169 -0.78560961]
 [-0.45425502 -1.14631534]
 [ 1.12139627  0.49643985]
 [ 0.61005958  0.58658564]
 [ 0.65656094 -0.7188364 ]]
[[-1.07558661 -0.90516512]
 [-1.45245481  1.3316412 ]
 [ 0.290075   -1.12393254]
 [ 1.23244214  1.11879751]]
[[4.98303832 2.99631132 3.90771005 1.01453877]
 [3.9922774  0.91805774 4.01290442 1.01257351]
 [1.00284911 1.00934179 2.81590131 4.98466882]
 [1.01578568 1.41662358 2.11080402 3.99987731]
 [3.54493065 1.00826738 4.98903792 3.99422334]]


In [2]:
#矩阵分解R=PQ，推荐SVD方法
import pandas as pd
import numpy as np 
from math import sqrt  
from sklearn.preprocessing import LabelEncoder

#矩阵分解函数
# @INPUT:
  #R     : a matrix to be factorized, dimension N x M
  #P     : an initial matrix of dimension N x K
  #Q     : an initial matrix of dimension M x K
  #K     : the number of latent features
  #steps : the maximum number of steps to perform the optimisation
  #alpha : the learning rate
  #beta  : the regularization parameter
#@OUTPUT: the final matrices P and Q

def matrix_factorization(R, P, Q, K, steps, alpha=0.0002, beta=0.002):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

#1.数据集处理
data = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False, iterator =True)
data = data.get_chunk(100)
#将userId和movieId全部标准编号
data_rating = data[['rating']]
le = LabelEncoder()
data = data[['userId','movieId']].apply(le.fit_transform)
data = pd.concat([data,data_rating],axis=1)
#抽样10%比例测试
test = data.sample(frac=0.1)
#2.建立评分矩阵
uNum = data['userId'].max()+1
iNum = data['movieId'].max()+1
R = np.zeros((uNum, iNum))#转成R矩阵，非常稀疏
for index, row in data.iterrows(): # 获取每行的值
    R[int(row['userId'])][int(row['movieId'])] = row['rating']
#3.SVD分解，随机梯度下降求解P、Q，并计算测试集均方误差
print ("%3s%20s%20s" % ('K','steps','RMSE'))
for K in [30,50,80,100]:#隐因子
    for steps in [2,100,200]:#迭代次数
        P = np.random.rand(uNum,K)
        Q = np.random.rand(iNum,K)
        #SVD分解
        nP, nQ = matrix_factorization(R, P, Q, K,steps)
        nR = np.dot(nP, nQ.T)  
        #RMSE评估   
        squaredError = []
        for index, row in test.iterrows(): # 获取每行的值
            pRating=nR[int(row['userId'])][int(row['movieId'])] #获取预测值
            TRating=row['rating']
            error=TRating-pRating
            squaredError.append(error * error)
        RMSE =sqrt(sum(squaredError) / len(squaredError))#均方根误差RMSE 
        print ("%3d%20d%20.6f" % (K,steps,RMSE))

  K               steps                RMSE
 30                   2            2.132338
 30                 100            0.292978
 30                 200            0.339896
 50                   2            2.940437
 50                 100            0.681469
 50                 200            0.294894
 80                   2            2.717870
 80                 100            0.644332
 80                 200            0.363208
100                   2            2.092428
100                 100            0.379074
100                 200            0.414947


In [18]:
import numpy as np
import pandas as pd
import math
from collections import defaultdict

#loading Movielen dataset , ml-20m
#trainset = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False)#, iterator =True)
#trainset shape is:20000263 rows and 4 columns,have 138493 users
#print ('trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
#Loading the  MovienLen dataset, ml-1m
def load_data_as_list(filename):
    data_list = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            data_list.append([user, item, rating])
            line = f.readline()
    return data_list
#user:1-6040, item:1-3706, rating:1-5
trainset = load_data_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.train.rating")
trainset = pd.DataFrame(trainset,columns=["u","i","r"])
print ('trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
testrate = load_data_as_list("/data/fjsdata/ctKngBase/ml/ml-1m.test.rating")
testrate = pd.DataFrame(testrate,columns=["u","i","r"])
print ('testrate shape is:%d rows and %d columns'%(testrate.shape[0],testrate.shape[1]))

trainset shape is:994169 rows and 3 columns
testrate shape is:6040 rows and 3 columns
